In [20]:
pip install ipywidgets jinja2 ipython

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from jinja2 import Environment, select_autoescape, FileSystemLoader, meta
from ipywidgets import widgets
from IPython.display import display, display_markdown
from IPython.display import Markdown
from jinja2 import Environment, FileSystemLoader, select_autoescape, meta, nodes
from jinja2.visitor import NodeVisitor

In [116]:

def get_default_env():
    return Environment(
        loader=FileSystemLoader('./templates'),
        autoescape=select_autoescape()
    )
env = get_default_env()

In [135]:


class DefaultValueExtractor(NodeVisitor):
    """Extract default values from Jinja2 AST"""
    
    def __init__(self):
        self.defaults = {}
        self.assignments = {}
    
    def visit_Filter(self, node):
        """Visit filter nodes to find | default() usage"""
        if isinstance(node.node, nodes.Name) and node.name in ('default', 'd'):
            var_name = node.node.name
            
            # Extract the default value from filter arguments
            if node.args:
                if isinstance(node.args[0], nodes.Const):
                    self.defaults[var_name] = node.args[0].value
                elif isinstance(node.args[0], nodes.Name):
                    self.defaults[var_name] = f"${node.args[0].name}"  # Reference to another variable
        
        # Continue visiting child nodes
        self.generic_visit(node)
    
    def visit_Assign(self, node):
        """Visit assignment nodes to find {% set var = var | default('value') %}"""
        if (isinstance(node.target, nodes.Name) and 
            isinstance(node.node, nodes.Filter) and 
            node.node.name in ('default', 'd')):
            
            var_name = node.target.name
            if (isinstance(node.node.node, nodes.Name) and 
                node.node.node.name == var_name and 
                node.node.args):
                
                if isinstance(node.node.args[0], nodes.Const):
                    self.assignments[var_name] = node.node.args[0].value
                elif isinstance(node.node.args[0], nodes.Add):
                    if isinstance(node.node.args[0].left, nodes.Const):
                        left = node.node.args[0].left.value
                    else:
                        left = '${' + node.node.args[0].left.name + '}'
                    if isinstance(node.node.args[0].right, nodes.Const):
                        right = node.node.args[0].right.value
                    else:
                        right = '${' + node.node.args[0].right.name + '}'


                    
                    self.assignments[var_name] = f"{left}{right}"

        
        self.generic_visit(node)

def extract_defaults_ast(template_content, env):
    """Extract defaults using AST parsing"""
    try:
        ast = env.parse(template_content)
        extractor = DefaultValueExtractor()
        extractor.visit(ast)
        
        # Combine direct defaults and assignments
        all_defaults = {**extractor.defaults, **extractor.assignments}
        return all_defaults
    except Exception as e:
        print(f"AST parsing failed: {e}")
        return {}
    
def get_template_defaults(template_path, env=None):
    """
    Get all default values for a template
    
    Args:
        template_path: Path to the template file
        env: Jinja2 environment (optional)
    
    Returns:
        dict: Variable names mapped to their default values
    """
    if env is None:
        env = get_default_env()
    
    try:
        # Get template source
        template_source = env.loader.get_source(env, template_path)[0]
        
        # Try AST method first (more accurate)
        defaults = extract_defaults_ast(template_source, env)

        required_vars = meta.find_undeclared_variables(env.parse(template_source))
        defaults.update({var: None for var in required_vars if var not in defaults})

        
        return defaults
        
    except Exception as e:
        print(f"Error extracting defaults from {template_path}: {e}")
        return {}

In [118]:
template = env.get_template('pyproject.toml.jinja')
template

<Template 'pyproject.toml.jinja'>

In [140]:
get_template_defaults('pyproject.toml.jinja', env)

{'PROJECT_NAME': 'python-boilerplate',
 'PACKAGE_NAME': 'python_boilerplate',
 'AUTHOR_NAME': 'RJ',
 'AUTHOR_EMAIL': 'rjlucas92@gmail.com',
 'GITHUB_REPO_URL': 'https://github.com/rjlucas92/${PROJECT_NAME}'}

In [101]:
env.loader.get_source(env, 'pyproject.toml.jinja')

('[build-system]\nrequires = ["setuptools>=64.0", "wheel"]\nbuild-backend = "setuptools.build_meta"\n\n[project]\nname = "{{PROJECT_NAME | default(\'python-boilerplate\')}}"\ndynamic = ["version"]\nauthors = [\n    {name = "{{AUTHOR_NAME}}", email = "{{AUTHOR_EMAIL}}"},\n]\n# Optional: Add a project description\ndescription = ""\nreadme = "README.md"\nrequires-python = ">=3.13"\nlicense = {text = "MIT"}\nclassifiers = [\n    "Development Status :: 3 - Alpha",\n    "Intended Audience :: Developers",\n    "License :: OSI Approved :: MIT License",\n    "Programming Language :: Python :: 3",\n    "Programming Language :: Python :: 3.13",\n    "Operating System :: OS Independent",\n]\ndependencies = [\n    # Add your package dependencies here\n]\n\n[project.urls]\n"Homepage" = "{{GITHUB_REPO_URL}}"\n"Bug Tracker" = "{{GITHUB_REPO_URL}}/issues"\n"Repository" = "{{GITHUB_REPO_URL}}"\n\n[tool.setuptools.dynamic]\nversion = {attr = "{{PACKAGE_NAME}}.version.__VERSION__"}\n\n[tool.setuptools.pac

In [102]:
env = Environment(
    loader=FileSystemLoader('./templates'),
    autoescape=select_autoescape()
)
meta.find_undeclared_variables(env.parse(env.loader.get_source(env, 'pyproject.toml.jinja')[0]))

{'AUTHOR_EMAIL',
 'AUTHOR_NAME',
 'GITHUB_REPO_URL',
 'PACKAGE_NAME',
 'PROJECT_NAME'}

In [103]:
env.list_templates(extensions=['jinja'])

['pyproject.toml.jinja']

In [37]:
import functools

In [ ]:
def parse_form(template, input_widgets):
    template_vars = {
        w.description.strip().strip(':'): w.value.strip() or w.placeholder
        for w in input_widgets if isinstance(w, widgets.Text)
    }
    print(template.render(**template_vars))

In [144]:
env = Environment(
    loader=FileSystemLoader('.'),
    autoescape=select_autoescape()
)

for t_path in env.list_templates(extensions=['jinja']):
    title = Markdown(f"# Template: `{t_path}`")
    # variables = meta.find_undeclared_variables(env.parse(env.loader.get_source(env, t_path)))
    variables = get_template_defaults(t_path, env)
    inputs = [widgets.Text(
        value='',
        placeholder=f"Ex: {default_val}" if default_val else '',
        description=f'{var_name}:',
        disabled=False,
        style={'description_width': 'initial'},
            layout=widgets.Layout(
        width='auto',    # Auto width
        flex='1 1 auto'  # Flexible sizing
    ),
    ) for var_name, default_val in variables.items()]
    inputs.append(widgets.Button(
        description='Render',
        button_style='success',
        icon='check',
        layout=widgets.Layout(width='auto'),
        style={'description_width': 'initial'},
        on_click=functools.partial(parse_form, template=env.get_template(t_path), input_widgets=inputs
    )))
    display(title, *inputs)
    

# Template: `templates/pyproject.toml.jinja`

Text(value='', description='PROJECT_NAME:', layout=Layout(flex='1 1 auto', width='auto'), placeholder='Ex: pyt…

Text(value='', description='PACKAGE_NAME:', layout=Layout(flex='1 1 auto', width='auto'), placeholder='Ex: pyt…

Text(value='', description='AUTHOR_NAME:', layout=Layout(flex='1 1 auto', width='auto'), placeholder='Ex: RJ',…

Text(value='', description='AUTHOR_EMAIL:', layout=Layout(flex='1 1 auto', width='auto'), placeholder='Ex: rjl…

Text(value='', description='GITHUB_REPO_URL:', layout=Layout(flex='1 1 auto', width='auto'), placeholder='Ex: …

Button(button_style='success', description='Render', icon='check', layout=Layout(width='auto'), style=ButtonSt…

In [12]:
w = widgets.Text(
    value='',
    placeholder='Type something',
    description='Author Email:',
    disabled=False,
    style={'description_width': 'initial'}
)
display(*[w])

Text(value='', description='Author Email:', placeholder='Type something', style=TextStyle(description_width='i…

In [34]:
help("a".strip)

Help on built-in function strip:

strip(chars=None, /) method of builtins.str instance
    Return a copy of the string with leading and trailing whitespace removed.

    If chars is given and not None, remove characters in chars instead.



In [36]:
inputs[0].description.strip().strip(':')

'PACKAGE_NAME'

In [126]:
env = Environment(
    loader=FileSystemLoader('./templates'),
    autoescape=select_autoescape()
)
print(env.get_template('pyproject.toml.jinja').render())

[build-system]
requires = ["setuptools>=64.0", "wheel"]
build-backend = "setuptools.build_meta"

[project]
name = "python-boilerplate"
dynamic = ["version"]
authors = [
    {name = "RJ", email = "rjlucas92@gmail.com"},
]
# Optional: Add a project description
description = ""
readme = "README.md"
requires-python = ">=3.13"
license = {text = "MIT"}
classifiers = [
    "Development Status :: 3 - Alpha",
    "Intended Audience :: Developers",
    "License :: OSI Approved :: MIT License",
    "Programming Language :: Python :: 3",
    "Programming Language :: Python :: 3.13",
    "Operating System :: OS Independent",
]
dependencies = [
    # Add your package dependencies here
]

[project.urls]
"Homepage" = "https://github.com/rjlucas92/python-boilerplate"
"Bug Tracker" = "https://github.com/rjlucas92/python-boilerplate/issues"
"Repository" = "https://github.com/rjlucas92/python-boilerplate"

[tool.setuptools.dynamic]
version = {attr = "python_boilerplate.version.__VERSION__"}

[tool.setupto